In [13]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git -t ../../..

In [14]:
import os
import sagemaker
from sagemaker.mxnet import MXNet

In [15]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
!rm -rf sourcedir.tar.gz
!tar czvf sourcedir.tar.gz *

In [ ]:
source_dir = 's3://{0}/{1}/source/'.format(bucket, 'gluonts')
!aws s3 cp sourcedir.tar.gz $source_dir
source_dir = '{0}sourcedir.tar.gz'.format(source_dir)
!rm -f sourcedir.tar.gz

In [ ]:
train_dir = 's3://{0}/{1}/data/train'.format(bucket, 'gluonts')
!aws s3 cp ../../../all-in-one-ai-sample-data/gluonts/train/ $train_dir --recursive

In [28]:
job_name = None
entry_point = 'train.py'
git_config = None
role = role
hyperparameters = {
    'algo-name': 'DeepAR', 
    'freq': '1M', 
    'prediction-length': 2*12, 
    'context-length': 20*12, 
    'epochs': 200, 
    'batch-size': 2048  , 
    'num-batches-per-epoch': 2
}
framework_version = '1.9.0'
py_version = 'py38'
instance_type = 'ml.g4dn.16xlarge'
instance_count = 1
inputs = {
    'dataset': train_dir
}

In [29]:
estimator = MXNet(
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    debugger_hook_config = False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [ ]:
estimator.fit(inputs, job_name = job_name)

In [ ]:
training_job_name = estimator.latest_training_job.name

In [ ]:
model_name = None
model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)
entry_point = 'inference.py'
framework_version = '1.9.0'
py_version = 'py38'
model_environment = {
    'freq': '1M'
}

In [ ]:
from sagemaker.mxnet.model import MXNetModel

model = MXNetModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [ ]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [ ]:
from os import listdir
from os.path import isfile, join

sample_dir ='../../../all-in-one-ai-sample-data/gluonts/inference/sample'
files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

inputs  = open(join(sample_dir, files[0]), 'r').read()

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(
    {
        'inputs': json.loads(inputs)
    }
)

In [ ]:
predictor.delete_endpoint()